# Use Case 1: Defibrillator data from out-of-hospital cardiac arrest case

<span style = "font-size:20px">This notebook illustrates the usage of the `vitabel` class to visualize, annotate and process time-series data from the medical field. Please find the detailed, searchable documentation here: 
[![Documentation Status](https://readthedocs.org/projects/vitabel/badge/?version=latest)](https://vitabel.readthedocs.io/en/latest/?badge=latest) <br>
In this case we analyze data recorded by a defibrillator during a real-world out-of-hospital cardiac arrest case.
This notebook in particular demonstrates the capabilities of vitabel to **display** and **annote** data.
</span>


In [1]:
from vitabel import Vitals, Label

import bz2
from pathlib import Path
import tempfile
import shutil
from datetime import datetime
import pandas as pd

## 1) Load Data

<span style = "font-size:18px">An empty vitabel object called `case` is initiated and all channels from a defibrillator recording (in this case a ZOLL X-Series case, stored in a compressed file) are loaded and added to the case.</span>

In [2]:
case = Vitals()

compressed_defi_file = Path("data/ZOLL_test_case.json.bz2")
with tempfile.TemporaryDirectory() as tmpdir:
    defi_file = Path(tmpdir) / compressed_defi_file.stem
    with bz2.open(compressed_defi_file, "rb") as source:
        with open(defi_file, "wb") as dest:
            shutil.copyfileobj(source, dest)

    case.add_defibrillator_recording(defi_file)

<span style = "font-size:18px">Every vitable object as well as every channel and label holds a dictionary as `metadata`. In the following we add the usage of the case in this notebook to the metadata. </span>

In [3]:
usage_metadata = {
    "vitable publication" : {"user" : "ENTER YOUR NAME HERE",
                             "purpose" : "higlighting capabilities to display and annotate defibrillator data",
                             "remarks" : ["a usefull tool for researchers", "handling data of devices by other manufacturers should be integrated better in the future"]
                            }
}
case.metadata.update({"usage":usage_metadata})

print("\nThis are the current metadata stored:\n")
case.metadata


This are the current metadata stored:



{'Time_of_construction': '2025-06-12 20:17:45.251808',
 'Recording_files_added': ['/tmp/tmprzu_pmca/ZOLL_test_case.json'],
 'Saving_to_json_time': '',
 'usage': {'vitable publication': {'user': 'ENTER YOUR NAME HERE',
   'purpose': 'higlighting capabilities to display and annotate defibrillator data',
   'remarks': ['a usefull tool for researchers',
    'handling data of devices by other manufacturers should be integrated better in the future']}}}

<span style = "font-size:18px">We get an overview over all loaded channels by calling `case.info()`</span>

In [4]:
case.info()

,Name,Length,First Entry,Last Entry,Offset,File ID,Serial Nr,Model,source
0,defibrillations_Nr,1,1970-01-01 07:36:54.645000,1970-01-01 07:36:54.645000,0 days 00:00:00,ZOLL_test_case.json,AR00XXXXXXX,ZOLL - X Series,ZOLL - X Series
1,defibrillations_DefaultEnergy,1,1970-01-01 07:36:54.645000,1970-01-01 07:36:54.645000,0 days 00:00:00,ZOLL_test_case.json,AR00XXXXXXX,ZOLL - X Series,ZOLL - X Series
2,defibrillations_DeliveredEnergy,1,1970-01-01 07:36:54.645000,1970-01-01 07:36:54.645000,0 days 00:00:00,ZOLL_test_case.json,AR00XXXXXXX,ZOLL - X Series,ZOLL - X Series
3,defibrillations_Impedance,1,1970-01-01 07:36:54.645000,1970-01-01 07:36:54.645000,0 days 00:00:00,ZOLL_test_case.json,AR00XXXXXXX,ZOLL - X Series,ZOLL - X Series
4,defibrillations_Current,1,1970-01-01 07:36:54.645000,1970-01-01 07:36:54.645000,0 days 00:00:00,ZOLL_test_case.json,AR00XXXXXXX,ZOLL - X Series,ZOLL - X Series
5,capnography,450750,1970-01-01 07:28:47.423000,1970-01-01 08:29:08.415000,0 days 00:00:00,ZOLL_test_case.json,AR00XXXXXXX,ZOLL - X Series,ZOLL - X Series
6,cpr_acceleration,1027500,1970-01-01 07:23:46.419000,1970-01-01 08:32:22.415000,0 days 00:00:00,ZOLL_test_case.json,AR00XXXXXXX,ZOLL - X Series,ZOLL - X Series
7,ecg_filtered,966000,1970-01-01 07:23:56.419000,1970-01-01 08:28:20.415000,0 days 00:00:00,ZOLL_test_case.json,AR00XXXXXXX,ZOLL - X Series,ZOLL - X Series
8,ecg_pads,1020750,1970-01-01 07:23:47.419000,1970-01-01 08:31:56.415000,0 days 00:00:00,ZOLL_test_case.json,AR00XXXXXXX,ZOLL - X Series,ZOLL - X Series
9,impedance,510000,1970-01-01 07:23:47.423000,1970-01-01 08:31:56.415000,0 days 00:00:00,ZOLL_test_case.json,AR00XXXXXXX,ZOLL - X Series,ZOLL - X Series


,Name,Length,First Entry,Last Entry,Offset


## 2) Process Data

<span style = "font-size:18px">We use integrated algorithms to extract **etCO₂** values and detect **ventilations** from the capnography signal.\
Additionally, we analyze the accelerometer data from the CPR feedback device and the ECG signal from the defibrillation pads to estimate the **probability of spontaneous circulation**.</span>

In [5]:
case.compute_etco2_and_ventilations()
case.predict_circulation()

/home/gpfs/o_orlob/.miniconda/envs/spa-ga/lib/python3.13/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/gpfs/o_orlob/.miniconda/envs/spa-ga/lib/python3.13/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RobustScaler from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## 3) Define new Label

<span style = "font-size:18px">We aim to manually **annotate** occurrences of **Return of Spontaneous Circulation (ROSC)**.\
Therefore we create an empty label and add it to the case. With `metadata` we can store miscellaneous data in the label inside a dictionary (`{}`).\
Further we can define the `plotstyle` by giving parameters in another dictionatry.\
<br>
`plot_type` defines the way data of the label are depicted in the plot. As the type is also dynamically adapted to the availabel data, we can explore the different types by adding labels with different data in the later generated plot.\
This can be done by activating and deactivating the checkboxes in various combinations.\
<br>
More details on Labels can be found [here](https://vitabel.readthedocs.io/en/latest/autoapi/vitabel/timeseries/index.html#vitabel.timeseries.Label).
</span>

In [6]:
ROSC_label = Label(
    name = "ROSC",
    time_index = [],
    data = [], 
    text_data = [], 
    metadata = {
        "Source": "manual annotation",
        "time_of_generation" : datetime.now()
    },
    plot_type = "combined",
    plotstyle = {"marker": "$\u2665$", "color": "red", "ms": 10, "linestyle": ""},

)

case.add_global_label(ROSC_label)

<span style = "font-size:18px">We see the newly added label (`ROSC`) by checking out all present labels in the case.</span>

In [7]:
case.get_label_names()

['rosc_prediction',
 'rosc_probability',
 'rosc_decision_function',
 'ROSC',
 'etco2_from_capnography',
 'ventilations_from_capnography',
 'cc_periods']

## 4) Interactively Plot Data

<span style = "font-size:18px">We provide a setup for the interactive plot.\
The `channels` to be plottted are defined as lists in a list (`[[],[]]`). Each of the inner lists define a new subplot.\
The `labels` to be plotted on top are defined by another lists in a list.\
Subplots below the already defined ones are defined by `channel_overviews`. These subplots serve a special purpose by displaying the entire recording interval and highlighting the depicted subsegement in the upper subplots.\
<br>
Above the plot a menu is given to `Annotate`, `Align Timelines`, and deffines `Settings` of the plot.

</span>

In [11]:
plot = case.plot_interactive(
    channels = [
        ["cpr_acceleration"], 
        ["capnography"], 
        ["ecg_pads"], 
        []
    ],
    labels = [
        ["ROSC", "cc_periods"],
        ["etco2_from_capnography", "ROSC"],
        ["ROSC"],
        ["ROSC", "rosc_probability"],
    ],
    channel_overviews = [["cpr_acceleration"]],
    time_unit = "s",
    subplots_kwargs = {"figsize": (16.5, 8)},
)
plot

UnboundLocalError: cannot access local variable 'filtered_base_plotstyle' where it is not associated with a value

<span style = "font-size:18px">Show the added `data` to the ROSC label</span>

In [9]:
ROSC_label.get_data()

(TimedeltaIndex([], dtype='timedelta64[ns]', freq=None), None, None)

## 5) Store Data
<span style = "font-size:18px">The entire case can be sotred as a json file.</span>


In [10]:
case.save_data("case_1.json")

TypeError: Object of type datetime is not JSON serializable

<span style = "font-size:18px">Data of the added label can also be stored as a csv spreadsheet.</span>

In [ ]:
case.get_label("ROSC").to_csv()
pd.read_csv("ROSC.csv", index_col = 0)

<span style = "font-size:18px">Alternatively the added label can be returned as dictionary with all its features.</span>


In [ ]:
case.get_label("ROSC").to_dict()